In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

# conectar base de datos
conn = sql.connect('Data/movies2.db')
cur = conn.cursor()

# ver tablas disponibles en base de datos ###
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('f_ratings',)]

In [5]:
df = pd.read_sql("SELECT * FROM f_ratings", conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       45425 non-null  int64  
 1   movie_id      45425 non-null  int64  
 2   rating        45425 non-null  float64
 3   timestamp     45425 non-null  int64  
 4   movie_title   45425 non-null  object 
 5   movie_genres  45425 non-null  object 
 6   fecha_nueva   45425 non-null  object 
 7   movie_year    45425 non-null  int64  
 8   clean_title   45425 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 3.1+ MB


# <b>3. Sistema de recomendación filtro colaborativo</b>
Se identifican usuarios que han calificado las mismas películas que el usuario en las últimas semanas. Se recomiendan las películas vistas recientemente por estos usuarios.<br>
Este sistema identifica usuarios que han calificado las mismas películas que el usuario principal durante las últimas semanas. A partir de esa coincidencia, se recomiendan las 10 películas que esos usuarios han visto recientemente y que el usuario aún no ha explorado. Al utilizar la actividad reciente de otros usuarios con gustos similares, el sistema ofrece recomendaciones más personalizadas y adaptadas a las preferencias actuales del usuario.<br> La frecuencia de actualización es semanal, asegurando que las recomendaciones reflejen las últimas interacciones tanto del usuario principal como de los usuarios que comparten preferencias similares. Además, el sistema se ajusta automáticamente para excluir las películas que el usuario ya ha visto.

In [15]:
ratings = pd.read_sql("SELECT * FROM f_ratings where rating>0", conn)

# los datos deben ser leídos en un formato especial para surprise
reader = Reader(rating_scale=(0.5,5))   # la escala de la calificación
# las columnas deben estar en orden estándar: user item rating
data = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)

In [18]:
# Lista con los modelos
models = [KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

# for para probar varios modelos
model = models[1]
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.643621,0.846066,0.316172,1.907523
knns.KNNWithZScore,0.648014,0.854598,0.292469,1.697138
knns.KNNWithMeans,0.651384,0.855646,0.235258,1.589296
knns.KNNBasic,0.690748,0.907492,0.306231,1.421164


<i>KNNBaseline</i>: calcula el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación,<br>se ecoge este modelo ya que tiene el menor valor tanto en MAE como en RMSE, lo que indica que, en promedio, este modelo realiza las predicciones con menor error en comparación con los otros modelos.